In [1]:
# !pip install --user implicit

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import implicit

In [3]:
train_df = pd.read_csv('train_recommender.csv')
train_df.shape

(330575, 7)

In [4]:
test_df = pd.read_csv('test_recommender.csv')
test_df.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [5]:
test_df = test_df.drop(['Country','InvoiceDate', 'InvoiceNo'],axis =1)

In [6]:
test_df =test_df.dropna()

In [7]:
test_df.columns
test_df.shape

(103097, 4)

In [8]:
test_df.loc[test_df.Quantity <0, 'Quantity'] = 3

In [9]:
test_df.loc[test_df.Quantity > 400,'Quantity']= 3

In [10]:
# UnitPrice =0 doesnot make any sense, replacing such values with mean 
test_df.loc[test_df.UnitPrice <=0.07,'UnitPrice']= 2.08

In [11]:
test_df.loc[test_df.UnitPrice >= 20,'UnitPrice']= 2.08

In [12]:
test_df['Value']= test_df['Quantity']* test_df['UnitPrice']

In [13]:
test_df.nlargest(10,'UnitPrice')

,CustomerID,Quantity,StockCode,UnitPrice,Value
11788,117279,2,22788F,19.96,39.92
12900,363726,1,22788F,19.96,19.96
13879,397296,1,22120W,19.96,19.96
14169,316377,1,22768C,19.96,19.96
14322,76419,1,22776E,19.96,19.96
15000,232668,1,22487O,19.96,19.96
15392,440856,1,22776E,19.96,19.96
15853,508185,1,22968N,19.96,19.96
18302,358236,1,22120W,19.96,19.96
18317,358236,1,22487O,19.96,19.96


In [14]:
test_customer = test_df['CustomerID'].drop_duplicates().tolist()
train_customer = train_df['CustomerID'].drop_duplicates().tolist()

In [15]:
test_df['StockCode'] = test_df['StockCode'].astype("category")
d= dict(enumerate(test_df['StockCode'].cat.categories))
test_df['product_id']= test_df['StockCode'].cat.codes
print(len(d))
test_df['CustomerID'] = test_df['CustomerID'].astype("category")
test_df['cust_id']= test_df['CustomerID'].cat.codes

3522


In [16]:
# print(test_df[['product_id','cust_id']])

In [17]:
#user_item matrix
item_user= sparse.csr_matrix((test_df['Value'].astype(float), (test_df['product_id'], test_df['cust_id'])))
user_item = sparse.csr_matrix((test_df['Value'].astype(float),(test_df['cust_id'], test_df['product_id'])))
item_user.shape

(3522, 628)

In [18]:
#model = implicit.als.AlternatingLeastSquares(factors=15,regularization=0.01, iterations=20)

In [19]:
model =implicit.bpr.BayesianPersonalizedRanking(factors= 10, learning_rate=0.01, regularization = 0.01, iterations =20)

In [20]:
import os 
try: 
    os.environ["OMP_NUM_THREADS"] = "1" 
    import numpy 
finally: 
   del os.environ["OMP_NUM_THREADS"] 

In [21]:
#Calculating the confidence
alpha_val = 30
data_conf = (item_user * alpha_val).astype('double')

In [22]:
#Fit the model
model.fit(data_conf)

#### Creating user recommendation

In [23]:
#creating recommendation for all users in the training set
#recommended =model.recommend_all(user_items = user_item, N = 350, recalculate_user= True, filter_already_liked_items =False)
#recommended

In [24]:
#creating recommendation for all users in the training set
recommended =model.recommend_all(user_items = user_item, N = 315, filter_already_liked_items =False)
recommended

array([[1245,  154, 3137, ..., 1607,  460,  475],
       [1245,  154, 1756, ...,  148,  868, 2138],
       [1245,  154, 3120, ..., 1287, 1286,   18],
       ...,
       [1245,  154, 1756, ..., 2121,  462,  578],
       [1245,  154, 3137, ..., 2832, 2121, 1210],
       [1245,  154, 1756, ..., 1749, 1031, 2239]], dtype=int32)

In [25]:
recommended.shape

(628, 315)

In [26]:
result=np.vectorize(d.get)(recommended)
result

array([['22423U', '20724F', '85123AY', ..., '22804V', '21314Q', '21340G'],
       ['22423U', '20724F', '22960K', ..., '20713F', '21974Q', '23370K'],
       ['22423U', '20724F', '85099BJ', ..., '22468F', '22467G',
        '15056ND'],
       ...,
       ['22423U', '20724F', '22960K', ..., '23349J', '21317B', '21498M'],
       ['22423U', '20724F', '85123AY', ..., '84596BQ', '23349J',
        '22381J'],
       ['22423U', '20724F', '22960K', ..., '22952P', '22180L', '23485W']],
      dtype='<U8')

In [27]:
df = pd.DataFrame()
df['CustomerID']= test_customer
df['Items'] = result.reshape(628,315).tolist()

In [28]:
#storing the submission file
submission = df[['CustomerID','Items']]
submission.to_csv("RecommenderfinalSubmission.csv",index= False)
submission.head()

,CustomerID,Items
0,127269,"[22423U, 20724F, 85123AY, 22960K, 22457C, 8487..."
1,227268,"[22423U, 20724F, 22960K, 85123AY, 22727B, 2245..."
2,327267,"[22423U, 20724F, 85099BJ, 84879M, 22457C, 2272..."
3,162738,"[22423U, 85123AY, 22457C, 20724F, 22960K, 2103..."
4,252747,"[22423U, 20724F, 22960K, 85123AY, 22457C, 8487..."
